In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csiro-biomass/sample_submission.csv
/kaggle/input/csiro-biomass/train.csv
/kaggle/input/csiro-biomass/test.csv
/kaggle/input/csiro-biomass/test/ID1001187975.jpg
/kaggle/input/csiro-biomass/train/ID2099464826.jpg
/kaggle/input/csiro-biomass/train/ID2037861084.jpg
/kaggle/input/csiro-biomass/train/ID1211362607.jpg
/kaggle/input/csiro-biomass/train/ID1853508321.jpg
/kaggle/input/csiro-biomass/train/ID193102215.jpg
/kaggle/input/csiro-biomass/train/ID698608346.jpg
/kaggle/input/csiro-biomass/train/ID1859251563.jpg
/kaggle/input/csiro-biomass/train/ID1880764911.jpg
/kaggle/input/csiro-biomass/train/ID853954911.jpg
/kaggle/input/csiro-biomass/train/ID1403107574.jpg
/kaggle/input/csiro-biomass/train/ID1781353117.jpg
/kaggle/input/csiro-biomass/train/ID384648061.jpg
/kaggle/input/csiro-biomass/train/ID1563418511.jpg
/kaggle/input/csiro-biomass/train/ID2125100696.jpg
/kaggle/input/csiro-biomass/train/ID482555369.jpg
/kaggle/input/csiro-biomass/train/ID638711343.jpg
/kaggle/input/c

In [2]:
import os
from pathlib import Path
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

# Config
TRAIN_CSV = Path("/kaggle/input/csiro-biomass/train.csv")
INPUT_ROOT = Path("/kaggle/input/csiro-biomass")
OUT_IMG_DIR = Path("/kaggle/working/train_aug")
OUT_CSV = Path("/kaggle/working/train_augmented.csv")
CREATE_ROT180 = True
CREATE_FLIP_X = True
CREATE_FLIP_Y = True

if OUT_CSV.exists() and OUT_IMG_DIR.exists() and any(OUT_IMG_DIR.iterdir()):
    print("Augmented CSV and images already exist. Skipping creation.")
else:
    print("Creating augmentations")
    OUT_IMG_DIR.mkdir(parents=True, exist_ok=True)

    df_orig = pd.read_csv(TRAIN_CSV)
    unique_paths = df_orig['image_path'].unique()

    created_images = {}
    def make_augmented_filename(orig_filename: str, suffix: str):
        p = Path(orig_filename)
        return f"{p.stem}{suffix}{p.suffix}"

    for img_rel in tqdm(unique_paths, desc="Creating augmented images"):
        src_path = INPUT_ROOT / img_rel
        if not src_path.exists():
            src_path = Path(img_rel)
            if not src_path.exists():
                raise FileNotFoundError(f"Could not find source image: {img_rel}")

        img = Image.open(src_path).convert("RGB")
        created_images[img_rel] = []

        if CREATE_ROT180:
            out_name = make_augmented_filename(img_rel, "_rot180")
            img_rot = img.rotate(180)
            img_rot.save(OUT_IMG_DIR / out_name, quality=95)
            created_images[img_rel].append(out_name)

        if CREATE_FLIP_X:
            out_name = make_augmented_filename(img_rel, "_flipx")
            img_fx = img.transpose(Image.FLIP_TOP_BOTTOM)
            img_fx.save(OUT_IMG_DIR / out_name, quality=95)
            created_images[img_rel].append(out_name)

        if CREATE_FLIP_Y:
            out_name = make_augmented_filename(img_rel, "_flipy")
            img_fy = img.transpose(Image.FLIP_LEFT_RIGHT)
            img_fy.save(OUT_IMG_DIR / out_name, quality=95)
            created_images[img_rel].append(out_name)

    aug_rows = []
    for _, row in tqdm(df_orig.iterrows(), total=len(df_orig), desc="Building augmented CSV rows"):
        img_rel = row['image_path']
        orig_sample_id = str(row['sample_id'])
        left, right = orig_sample_id.split("__", 1)
        base_image_id = left
        target_name = right
        for out_name in created_images[img_rel]:
            suffix = "_" + Path(out_name).stem.replace(Path(img_rel).stem, "")
            new_row = row.copy()
            new_image_id = f"{base_image_id}{suffix}"
            new_row['sample_id'] = f"{new_image_id}__{target_name}"
            # relative path in new CSV points to OUT_IMG_DIR basename (we will use 'train_aug/<name>')
            new_row['image_path'] = str(Path(OUT_IMG_DIR.name) / out_name)
            aug_rows.append(new_row)

    df_aug = pd.DataFrame(aug_rows)
    final_df = pd.concat([df_orig, df_aug], ignore_index=True)
    final_df.to_csv(OUT_CSV, index=False)
    print("Saved augmented CSV to:", OUT_CSV)
    print("Saved augmented images to:", OUT_IMG_DIR)
    print("Original rows:", len(df_orig), "Augmented rows added:", len(df_aug), "Final rows:", len(final_df))


Creating augmentations


Creating augmented images:   0%|          | 0/357 [00:00<?, ?it/s]

Building augmented CSV rows:   0%|          | 0/1785 [00:00<?, ?it/s]

Saved augmented CSV to: /kaggle/working/train_augmented.csv
Saved augmented images to: /kaggle/working/train_aug
Original rows: 1785 Augmented rows added: 5355 Final rows: 7140


In [3]:
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm.auto import tqdm
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pathlib
from pathlib import Path
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import random
import copy
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold


print("Done")

Done


In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
torch.backends.cudnn.benchmark = True

USE_AUGMENTED = True   # True if you have run the previour cell
AUG_CSV_PATH = Path("/kaggle/working/train_augmented.csv")
AUG_IMG_DIR  = Path("/kaggle/working/train_aug")

TRAIN_CSV = "/kaggle/input/csiro-biomass/train.csv"
TEST_CSV = "/kaggle/input/csiro-biomass/test.csv"
TRAIN_IMG_ROOT = "/kaggle/input/csiro-biomass/train"
TEST_IMG_ROOT  = "/kaggle/input/csiro-biomass/test"

# hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 16
LR = 3e-4
EPOCHS = 30 # change to 30 later
N_FOLDS = 5 # change to 5 later
TAB_DIM = 8

TARGET_NAMES = ["Dry_Clover_g","Dry_Dead_g","Dry_Green_g","Dry_Total_g","GDM_g"]
target_weights = {"Dry_Green_g":0.1,"Dry_Dead_g":0.1,"Dry_Clover_g":0.1,"GDM_g":0.2,"Dry_Total_g":0.5}


Device: cuda


In [5]:
# if USE_AUGMENTED:
df_raw = pd.read_csv(AUG_CSV_PATH)
IMG_ROOT = Path.cwd() / AUG_IMG_DIR.name

def fix_path(p):
    p = str(p)
    if p.startswith("train_aug/"):
        return str(IMG_ROOT / Path(p).name)
    return str(Path(TRAIN_IMG_ROOT) / Path(p).name)

df_raw["image_path"] = df_raw["image_path"].apply(fix_path)

df_raw['image_path'] = df_raw['image_path'].astype(str)
df_raw['image_file_stem'] = df_raw['image_path'].apply(lambda p: Path(p).stem)
df_raw['orig_image_id'] = df_raw['image_file_stem'].str.split("_").str[0]

meta_cols = ["image_path","Sampling_Date","State","Species","Pre_GSHH_NDVI","Height_Ave_cm"]
wide_targets = df_raw.pivot_table(index="image_file_stem", values="target", columns="target_name", aggfunc='first').reset_index()
meta = df_raw.groupby("image_file_stem").first()[meta_cols].reset_index()
df = meta.merge(wide_targets, on="image_file_stem", how="left")

orig_map = df_raw.groupby("image_file_stem")["orig_image_id"].first().reset_index()
df = df.merge(orig_map, on="image_file_stem", how="left")

df = df.reset_index(drop=True)
df["global_idx"] = np.arange(len(df))

print("Per-file samples (including augmentations):", len(df))

Per-file samples (including augmentations): 1428


In [6]:
def add_date_features(df, date_col="Sampling_Date"):
    dates = pd.to_datetime(df[date_col], errors="coerce")
    doy = dates.dt.dayofyear.fillna(1).astype(int)
    df = df.copy()
    df["date_sin"] = np.sin(2*np.pi*doy/365.25)
    df["date_cos"] = np.cos(2*np.pi*doy/365.25)
    return df

df = add_date_features(df)

In [7]:
n = len(df)
m = len(TARGET_NAMES)

targets = np.zeros((n, m), dtype=np.float32)
mask = np.zeros((n, m), dtype=np.float32)

for i, row in df.iterrows():
    for j, t in enumerate(TARGET_NAMES):
        val = row.get(t, np.nan)
        if pd.notna(val):
            targets[i,j] = float(val)
            mask[i,j] = 1.0
        else:
            targets[i,j] = 0.0
            mask[i,j] = 0.0

In [8]:
test_raw = pd.read_csv(TEST_CSV)
test_raw['image_path'] = test_raw['image_path'].astype(str)
test_raw['image_file_stem'] = test_raw['image_path'].apply(lambda p: Path(p).stem)
test_images = test_raw.groupby("image_file_stem").first().reset_index()[["image_file_stem","image_path"]]
print("Test images count:", len(test_images))

Test images count: 1


In [9]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.1,0.1,0.1,0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [10]:
class FusionDataset(Dataset):
    def __init__(self, df_subset, tab_arr, target_arr=None, mask_arr=None, transform=None, img_root=None, return_index=False):
        self.df = df_subset.reset_index(drop=True)
        self.tab = torch.tensor(tab_arr, dtype=torch.float32)
        self.targets = torch.tensor(target_arr, dtype=torch.float32) if target_arr is not None else None
        self.mask = torch.tensor(mask_arr, dtype=torch.float32) if mask_arr is not None else None
        self.transform = transform
        self.img_root = Path(img_root) if img_root is not None else None
        self.return_index = return_index

    def __len__(self): return len(self.df)

    def _resolve(self, p):
        p = Path(p)
        if p.exists(): return p
        if self.img_root is not None:
            cand = self.img_root / p.name
            if cand.exists(): return cand
        return p

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self._resolve(row["image_path"])
        img = Image.open(str(img_path)).convert("RGB")
        if self.transform: img = self.transform(img)
        tab = self.tab[idx]
        if self.targets is not None:
            if self.return_index:
                return img, tab, self.targets[idx], self.mask[idx], int(row['global_idx'])
            return img, tab, self.targets[idx], self.mask[idx]
        else:
            if self.return_index:
                return img, tab, int(row['global_idx'])
            return img, tab, row["image_file_stem"]

class TestImageDataset(Dataset):
    def __init__(self, df, tab_arr, transform=None, img_root=None):
        self.df = df.reset_index(drop=True)
        self.tab = torch.tensor(tab_arr, dtype=torch.float32)
        self.transform = transform
        self.img_root = Path(img_root) if img_root is not None else None
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(row["image_path"])
        if not img_path.exists() and self.img_root is not None:
            img_path = self.img_root / img_path.name
        img = Image.open(str(img_path)).convert("RGB")
        if self.transform: img = self.transform(img)
        return img, self.tab[idx], row["image_file_stem"]

class FusionMultiOutputModel(nn.Module):
    def __init__(self, tab_dim, n_targets=len(TARGET_NAMES), freeze_backbone=False, pretrained=False):
        super().__init__()
        from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
        
        try:
            weights = ConvNeXt_Tiny_Weights.DEFAULT if pretrained else None
            backbone = convnext_tiny(weights=weights)
        except:
            backbone = convnext_tiny(weights=None)
        
        # Remove the classifier head
        backbone.classifier = nn.Identity()
        self.backbone = backbone
        
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False
        
        # Feature dimensions
        image_feat_dim = 768  # ConvNeXt-Tiny outputs 768-dim features
        hidden1 = max(32, tab_dim*2)
        
        # Tabular MLP
        self.tab_mlp = nn.Sequential(
            nn.Linear(tab_dim, hidden1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden1),
            nn.Linear(hidden1, 64),
            nn.ReLU()
        )
        
        # Fusion MLP
        fusion_dim = image_feat_dim + 64
        self.shared = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3)
        )
        
        # Separate heads for each target
        self.heads = nn.ModuleList([
            nn.Sequential(nn.Linear(512, 128), nn.ReLU(), nn.Linear(128, 1))
            for _ in range(n_targets)
        ])
    
    def forward(self, img, tab):
        img_feat = self.backbone(img)  # [B, C, H, W]
        if img_feat.dim() == 4:
            img_feat = torch.flatten(img_feat, 1)  # [B, C*H*W]
            # Or better: use adaptive avg pooling to 1x1
            # img_feat = torch.nn.functional.adaptive_avg_pool2d(img_feat, 1).view(img_feat.size(0), -1)
        tab_feat = self.tab_mlp(tab)
        x = torch.cat([img_feat, tab_feat], dim=1)
        x = self.shared(x)
        out = torch.cat([h(x) for h in self.heads], dim=1)
        return out



In [11]:
def build_tabular_from_df_local(given_df, enc=None, scaler=None):
    n = len(given_df)
    return np.zeros((n, TAB_DIM), dtype=np.float32)

def masked_mse(preds, targets, mask):
    diff = (preds - targets) * mask
    denom = mask.sum()
    if denom == 0:
        return torch.tensor(0.0, device=preds.device, dtype=preds.dtype)
    return (diff**2).sum() / denom

def weighted_r2(y_true, y_pred, names):
    w = np.array([target_weights[n] for n in names], dtype=float)
    mu = np.sum(w * y_true) / np.sum(w)
    ss_res = np.sum(w * (y_true - y_pred)**2)
    ss_tot = np.sum(w * (y_true - mu)**2)
    if ss_tot == 0: return 0.0
    return 1.0 - ss_res / ss_tot

In [12]:
test_preds_accum = np.zeros((len(test_images), len(TARGET_NAMES)), dtype=np.float32)
test_counts = np.zeros(len(test_images), dtype=np.float32)
test_index_map = {row["image_file_stem"]: idx for idx, row in test_images.reset_index().iterrows()}

In [13]:
groups = df["orig_image_id"].values
gkf = GroupKFold(n_splits=N_FOLDS)

fold_no = 0
for train_idx, val_idx in gkf.split(df, df, groups=groups):
    fold_no += 1
    print(f"Fold {fold_no}/{N_FOLDS}")
    df_train = df.iloc[train_idx].reset_index(drop=True)
    df_val = df.iloc[val_idx].reset_index(drop=True)
    y_train_raw = targets[train_idx]; mask_train = mask[train_idx]
    y_val_raw = targets[val_idx]; mask_val = mask[val_idx]

    # Fit enc/scaler on train only
    enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    enc.fit(df_train[["State","Species"]].fillna("missing"))
    scaler = StandardScaler()
    scaler.fit(df_train[["Pre_GSHH_NDVI","Height_Ave_cm"]].fillna(0).values)

    X_tab_train = build_tabular_from_df_local(df_train, enc, scaler)
    X_tab_val = build_tabular_from_df_local(df_val, enc, scaler)

    # compute target means/std on train only
    t_means = np.zeros(m, dtype=np.float32); t_stds = np.ones(m, dtype=np.float32)
    for j in range(m):
        vals = y_train_raw[mask_train[:,j]==1, j]
        if len(vals) > 0:
            t_means[j] = vals.mean()
            t_stds[j]  = vals.std() if vals.std() > 0 else 1.0

    # normalize the global targets using train stats
    targets_norm = (targets - t_means.reshape(1,-1)) / t_stds.reshape(1,-1)
    y_tr = targets_norm[train_idx]; y_v = targets_norm[val_idx]

    # dataloaders
    train_ds = FusionDataset(df_train, X_tab_train, y_tr, mask_train, transform=train_transform, img_root=IMG_ROOT, return_index=False)
    val_ds = FusionDataset(df_val, X_tab_val, y_v, mask_val, transform=val_transform, img_root=IMG_ROOT, return_index=False)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=(DEVICE=="cuda"))
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=(DEVICE=="cuda"))


    # model, optimizer, scheduler
    tab_dim = X_tab_train.shape[1]
    model = FusionMultiOutputModel(tab_dim, n_targets=m).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=False)

    best_val_r2 = -9e9
    patience_counter = 0
    PATIENCE = 3 
    best_state = None

    # training epochs
    for epoch in range(1, EPOCHS+1):
        model.train()
        running_loss_sum = 0.0
        running_obs = 0.0
        train_iter = tqdm(train_loader, desc=f"Fold{fold_no} Ep{epoch} train", leave=False)
        for batch in train_iter:
            imgs, tabs, ys, masks_batch = batch
            imgs = imgs.to(DEVICE)
            tabs = tabs.to(DEVICE)
            ys = ys.to(DEVICE)
            masks_batch = masks_batch.to(DEVICE)
            
            preds = model(imgs, tabs)
            loss = masked_mse(preds, ys, masks_batch)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
            
            n_obs = float(masks_batch.sum().item())
            running_loss_sum += loss.item() * n_obs
            running_obs += n_obs
            train_iter.set_postfix({"batch_loss": f"{loss.item():.4f}"})
            
        train_loss = running_loss_sum / max(1e-8, running_obs)

        # validation
        model.eval()
        val_loss_sum = 0.0
        val_obs = 0.0
        all_true = []
        all_pred = []
        all_names = []
        
        val_iter = tqdm(val_loader, desc=f"Fold{fold_no} Ep{epoch} val  ", leave=False)
        with torch.no_grad():
            for batch in val_iter:
                imgs, tabs, ys, masks_batch = batch
                imgs = imgs.to(DEVICE)
                tabs = tabs.to(DEVICE)
                ys = ys.to(DEVICE)
                masks_batch = masks_batch.to(DEVICE)
                
                preds_norm = model(imgs, tabs)
                loss = masked_mse(preds_norm, ys, masks_batch)
                
                n_obs = float(masks_batch.sum().item())
                val_loss_sum += loss.item() * n_obs
                val_obs += n_obs

                preds_raw = preds_norm.cpu().numpy() * t_stds.reshape(1,-1) + t_means.reshape(1,-1)
                ys_raw = ys.cpu().numpy() * t_stds.reshape(1,-1) + t_means.reshape(1,-1)
                masks_np = masks_batch.cpu().numpy()
                
                for i in range(preds_raw.shape[0]):
                    for j, tname in enumerate(TARGET_NAMES):
                        if masks_np[i,j] == 1:
                            all_true.append(float(ys_raw[i,j]))
                            all_pred.append(float(preds_raw[i,j]))
                            all_names.append(tname)
                            
                val_iter.set_postfix({"batch_loss": f"{loss.item():.4f}"})

        val_loss = val_loss_sum / max(1e-8, val_obs)
        val_r2 = weighted_r2(np.array(all_true), np.array(all_pred), all_names) if len(all_true) else 0.0
        print(f"Fold{fold_no} Epoch {epoch} train_loss={train_loss:.6f} val_loss={val_loss:.6f} val_weightedR2={val_r2:.6f}")

        scheduler.step(val_loss)
        if val_r2 > best_val_r2:
            best_val_r2 = val_r2
            best_state = {k: v.cpu().clone() for k,v in model.state_dict().items()}
            patience_counter = 0
            print("  New best val weighted R2:", best_val_r2)
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"Early stopping triggered at epoch {epoch}. Best val R2={best_val_r2:.6f}")
                break

    
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"Fold {fold_no}: restored best model state (val weighted R2={best_val_r2:.6f})")
        
    # Test inference for this fold (accumulate)
    tabular_test = np.zeros((len(test_images), TAB_DIM), dtype=np.float32)

    
    test_ds = TestImageDataset(test_images, tabular_test, transform=val_transform, img_root=TEST_IMG_ROOT)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=(DEVICE=="cuda"))

    with torch.no_grad():
        for imgs, tabs, stems in tqdm(test_loader, desc=f"Fold{fold_no} Test infer"):
            imgs = imgs.to(DEVICE); tabs = tabs.to(DEVICE)
            preds_norm = model(imgs, tabs).cpu().numpy()
            preds_raw = preds_norm * t_stds.reshape(1,-1) + t_means.reshape(1,-1)
            preds_raw = np.clip(preds_raw, 0.0, None)
            for stem, vec in zip(stems, preds_raw):
                ti = test_index_map[stem]
                test_preds_accum[ti] += vec
                test_counts[ti] += 1

# Average test predictions across folds
test_counts = np.maximum(test_counts, 1)
test_preds_avg = test_preds_accum / test_counts.reshape(-1,1)

Fold 1/5


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Fold1 Ep1 train:   0%|          | 0/72 [00:00<?, ?it/s]

Fold1 Ep1 val  :   0%|          | 0/18 [00:00<?, ?it/s]

Fold1 Epoch 1 train_loss=0.970319 val_loss=0.908726 val_weightedR2=0.322057
  New best val weighted R2: 0.322057327511275


...

Fold5 Epoch 12 train_loss=0.815218 val_loss=1.130868 val_weightedR2=0.296105


Fold5 Ep13 train:   0%|          | 0/72 [00:00<?, ?it/s]

Fold5 Ep13 val  :   0%|          | 0/18 [00:00<?, ?it/s]

Fold5 Epoch 13 train_loss=0.814070 val_loss=1.105728 val_weightedR2=0.351277
Early stopping triggered at epoch 13. Best val R2=0.362892
Fold 5: restored best model state (val weighted R2=0.362892)


Fold5 Test infer:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
rows = []
for i,row in test_images.reset_index().iterrows():
    stem = row["image_file_stem"]
    for tname, val in zip(TARGET_NAMES, test_preds_avg[i]):
        rows.append({"sample_id": f"{stem}__{tname}", "target": float(val)})
submission = pd.DataFrame(rows)
submission.to_csv("./submission.csv", index=False)
print("Wrote submission.csv with", len(submission), "rows")


Wrote submission.csv with 5 rows
